# Predicting Sale Price for Ames Iowa Housing Department

By: Christopher Kuzemka: [Github]

## Problem Statement

My girlfirned loves to use Reddit. One of her favorite subreddits is "r/aww", a community dedicated forum largely consisting of cute animal and cute moments captured on video or on camera. A majority of the of their content   

## Executive Summary

 

## Table of Contents
[1.00 Data Loading](#1.00-Data-Loading)

[2.00 Data Cleaning and Analysis](#2.00-Data-Cleaning-and-Moderate-Analysis)

- [2.01 Quick Check](#2.01-Quick-Check)

- [2.02 Data Documentation Exploration](#2.02-Data-Documentation-Exploration)

- [2.03 Cleaning](#2.03-Cleaning)

- [2.04 Feature Engineering](#2.04-Feature-Engineering)

- [2.05 Dummifying Columns](#2.05-Dummifying-Columns)

- [2.06 Exploratory Data Analysis and Visualization](#2.06-Exploratory-Data-Analysis-and-Visualization)

[3.00 Mimic Data Transformation Onto Test Set](#3.00-Mimic-Data-Transformation-Onto-Test-Set)

[4.00 Machine Learning Modeling and Visulalization](#4.00-Machine-Learning-Modeling-and-Visulalization)

- [4.01 Model Preparation](#4.01-Model-Preparation)

- [4.02 Modeling](#4.02-Modeling)

- [4.03 Model Selection](#4.03-Model-Selection)

- [4.04 Model Evaluation](#4.04-Model-Evaluation)

[5.00 Conclusions](#5.00-Conclusions)

[6.00 Sources and References](#6.00-Sources-and-References)

# Link to DATA DICTIONARY

[DATA DICTIONARY](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt) 